In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
import yaml
import os



class Answer(BaseModel):
    title_en: str = Field(description="")
    title_ko: str = Field(description="")
    contents: str = Field(default="")
    detail_contents: str = Field(description="")
    urls: str = Field(description="")

class QandAChain:
    def __init__(self):
        base_path = os.getcwd() 
        # base_path = os.path.dirna/e(os.path.abspath(__file__))  # 현재 파일의 경로
        file_path = os.path.join(base_path, 'prompt', 'QandAPrompt.yaml')

        #with open(file_path, 'r') as f:
        #    self.__prompt_template = yaml.load(f, Loader=yaml.SafeLoader)['template']
        
        self.__prompt_template = """
        너는 여행 정보를 제공하는 AI야.

        사용자의 질문에 아래 JSON 형식으로 답변해 줘:

        {format_instructions}

        질문: {question}
        """

        self.__model = ChatOpenAI(
            temperature = 0.1,
            model_name="gpt-4o-mini",
            max_tokens = 2048,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        self.__parser = JsonOutputParser(pydantic_object=Answer)
        # self.__parser = StrOutputParser()

    def main_chain_invoke(self, question):
        prompt = PromptTemplate.from_template(self.__prompt_template)
        prompt = prompt.partial(format_instructions=self.__parser.get_format_instructions())

        chain = prompt | self.__model | self.__parser

        result = chain.invoke({"question": question})
        return result

In [13]:
if __name__ == "__main__":
    chain = QandAChain()
    result = chain.main_chain_invoke("LangChain이란?")
    print(result)

{'title_en': 'What is LangChain?', 'title_ko': 'LangChain이란?', 'contents': 'LangChain은 자연어 처리(NLP) 및 인공지능(AI) 모델을 활용하여 다양한 애플리케이션을 구축할 수 있도록 돕는 프레임워크입니다.', 'detail_contents': 'LangChain은 언어 모델을 사용하여 데이터 처리, 대화형 AI, 정보 검색, 자동화된 작업 등을 수행할 수 있는 도구와 라이브러리를 제공합니다. 이 프레임워크는 사용자가 쉽게 언어 모델을 통합하고, 다양한 데이터 소스와 상호작용할 수 있도록 설계되었습니다.', 'urls': 'https://langchain.readthedocs.io/'}
